In [1]:
import time
import csv as c
import numpy as np
from scipy.spatial import cKDTree
from sklearn.ensemble import RandomForestClassifier


In [7]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    if iteration == total: 
        print()



In [2]:
def getEigenN(matrix):
    cov = np.cov(matrix.T)
    val, vec = np.linalg.eig(cov)
    srt = val.argsort()[::-1]
    val = val[srt]
    vec = vec[:,srt]
    if (val[2] < 0):
        val[2] = 0
    val = val/sum(val)
    return val, vec

In [3]:
#建立分類器 （分地板 非地板）
#讀取之前存的features and tag
clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=4)
features = np.genfromtxt("features.csv",delimiter=",")
tags = np.genfromtxt("tags.csv",delimiter=",")
clf.fit(features, tags)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=4, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [4]:
#要分割的檔案（含位置）
objName = "WithGround.csv"

In [5]:
#KD Tree
testData = np.genfromtxt (objName, delimiter=",", usecols=(0,1,2))
testDataKD = cKDTree(testData, leafsize=50)

In [8]:
#計算輸入檔案的features
testFeatures = []
used = []

p1 = len(testData)/100
p2 = 0

printProgressBar(0, 100, prefix = 'Progress:', suffix = 'Complete', length = 50)
for i in range(len(testData)):
    neighborIndex = testDataKD.query_ball_point(testData[i], 1)
    
    
    if len(neighborIndex) >= 3:
        neighbors = testData[neighborIndex]
        val, vec = getEigenN(neighbors)
        vec = vec.T
        
        EV1,EV2,EV3 = val[0],val[1],val[2]
        VT1, VT2, VT3 = vec[0],vec[1],vec[2]
        
        for j in range(3):
            VT3[j] = abs(VT3[j])
        vertical = -(VT3[2]/sum(VT3) - 1)

        feature = [(EV1-EV2)/EV1, (EV2-EV3)/EV1, EV3/EV1, vertical]
        testFeatures.append(feature)
        used.append(i)
    else:
        pass

    if (i > p1):
        p2 += 1
        p1 += len(testData)/100
        printProgressBar(p2, 100, prefix = 'Progress:', suffix = 'Complete', length = 50)
printProgressBar(100, 100, prefix = 'Progress:', suffix = 'Complete', length = 50)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [9]:
#預測輸入的檔案
pred = clf.predict(testFeatures)

In [10]:
#存results.csv
with open("results.csv","w") as f:
    writer = c.writer(f)
    for i in range(len(used)):
        if pred[i] == 1:
            writer.writerow(testData[used[i]].tolist() + [255,0,0])
        else:
            writer.writerow(testData[used[i]].tolist() + [0,255,0])